In [1]:
from utils.load_dataset import *
from utils.custom_utils import *

In [2]:
model_path_var = "./models/checkpoint_train_virtual_and_real.pth"
model_path_vtr = "./models/checkpoint_train_real_over_virtual.pth"
model_path_r = "./models/checkpoint_train_real.pth"
model_path_v = "./models/checkpoint_train_virtual.pth"

In [3]:
def load_model(path):
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    # create a Faster R-CNN model without pre-trained
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False, pretrained_backbone=False)

    num_classes = 3 # wheat or not(background)

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features

    # replace the pre-trained model's head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    checkpoint = torch.load(path, map_location=device)
    model.load_state_dict(checkpoint['model'])
    # load the trained weights
    #model.load_state_dict(torch.load(path, map_location=device))
    model.eval()

    # move model to the right device
    model.to(device)
    return model, device

In [4]:
def take_prediction(prediction, threshold):
    boxes = prediction['boxes'].tolist()
    labels = prediction['labels'].tolist()
    scores = prediction['scores'].tolist()
    if len(boxes) == 0:
        return [([0,0,0,0],1,0.)]
    
    res = [t for t in zip(boxes,labels,scores) if t[2]>threshold]
    if len(res) == 0:
        res = [([0,0,0,0],1,0.)]
    return res

In [5]:
def visualize_prediction(dataset, list_imgs, model, device, path=None, thr=0.7):
    for l in list_imgs:
        img,target = dataset[l]
        with torch.no_grad():
            prediction = model([img.to(device)])
        p = take_prediction(prediction[0],thr)
        for bb,label,score in p:
            if label == 1:
                color = "green"
                text = f"no fallen: {score:.3f}"
            else:
                color = "red"
                text = f"fallen: {score:.3f}"
            x0,y0,x1,y1 = bb
            im = Image.fromarray(img.mul(255).permute(1, 2, 0).byte().numpy())
            draw = ImageDraw.Draw(im)
            draw.rectangle(((x0, y0),(x1,y1)), outline=color, width=3)
            draw.text((x0, y0), text)
        if path is None:
            ImageShow.show(im)
        else:
            im.save(f'snapshots/{path}{l}.png')

In [6]:
def classifier_performance(dataset, model, device, fallen_or_not, tr = 0.7):
    tn, tp, fn, fp = 0,0,0,0
    for im,target in tqdm(dataset):
        gt_labels = target['labels'].tolist()
        with torch.no_grad():
            prediction = model([im.to(device)])
        p = take_prediction(prediction[0], tr)
        pred_labels = [l for _,l,_ in p]
        len_gt_lab = len(gt_labels)
        len_pred_lab = len(pred_labels)
        fall_gt = [i for i in gt_labels if i==fallen_or_not]
        num_fall_gt = len(fall_gt)
        num_no_fall_gt = len_gt_lab - num_fall_gt

        fall_pred = [i for i in pred_labels if i==fallen_or_not] 
        num_fall_pred = len(fall_pred)
        num_no_fall_pred = len_pred_lab - num_fall_pred

        if num_fall_gt == num_fall_pred:
            tp += num_fall_gt
        elif num_fall_gt > num_fall_pred:
            tp += num_fall_pred
            fn += (num_fall_gt - num_fall_pred)
        else:
            tp += num_fall_gt
            fp += (num_fall_pred - num_fall_gt)
    return tp, fp, fn

# UP-FALL DETECTION

In [22]:
test_udf = load_data("test_ufd.txt")
test_d = FallenPeople(test_udf, "test_ufd", FallenPeople.valid_test_transform())

subset_indeces = [i for i in range(len(test_d)) if i%5==0]
test_dataset = Subset(test_d, subset_indeces)

In [43]:
no_fall = 0
fall = 0
for t in test_dataset:
    if t[1]['labels'][0] == 1:
        no_fall += 1
    elif t[1]['labels'][0] == 2:
        fall += 1
print(no_fall)
print(fall)

241
179


In [23]:
def collate_fn(batch):
        return tuple(zip(*batch))
    
test_data_loader = DataLoader(
      test_dataset,
      batch_size=4,
      shuffle=False,
      collate_fn=collate_fn
    )

In [16]:
model_var, device = load_model(model_path_var)
model_vtr, _ = load_model(model_path_vtr)
model_r, _ = load_model(model_path_r)
model_v, _ = load_model(model_path_v)
test_list = [random.randint(0,len(test_dataset)-1) for i in range(5)]

# ONLY REAL

In [24]:
evaluate(model_r, test_data_loader, device=device)

creating index...
index created!
Test:  [  0/105]  eta: 0:00:45  model_time: 0.3349 (0.3349)  evaluator_time: 0.0048 (0.0048)  time: 0.4304  data: 0.0713  max mem: 2030
Test:  [100/105]  eta: 0:00:01  model_time: 0.3223 (0.3231)  evaluator_time: 0.0056 (0.0057)  time: 0.3776  data: 0.0458  max mem: 2030
Test:  [104/105]  eta: 0:00:00  model_time: 0.3232 (0.3232)  evaluator_time: 0.0057 (0.0057)  time: 0.3794  data: 0.0465  max mem: 2030
Test: Total time: 0:00:39 (0.3807 s / it)
Averaged stats: model_time: 0.3232 (0.3232)  evaluator_time: 0.0057 (0.0057)
Accumulating evaluation results...
DONE (t=0.10s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.588
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.934
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.695
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=me

### FALL DETECTION

In [25]:
recall, precision, f1_score = [],[],[]
for thr in range(0,10,1):
    tp, fp, fn = classifier_performance(test_dataset, model_r, device, 2,float(thr/10))
    rec = tp / (tp+fn)
    prec = tp/ (tp+fp)
    f1_s = 2/((1/rec)+(1/prec))
    recall.append(f"{rec:.2f}")
    precision.append(f"{prec:.2f}")
    f1_score.append(f"{f1_s:.2f}")
    print(f"TP: {tp}\tFN: {fn}\tFP: {fp}")
    print(f"Recall: {rec:.2f}")
    print(f"Precision: {prec:.2f}")
    print(f"F1-score: {f1_s:.2f}")

100%|██████████| 420/420 [00:42<00:00,  9.93it/s]


TP: 179	FN: 0	FP: 1121
Recall: 1.00
Precision: 0.14
F1-score: 0.24


100%|██████████| 420/420 [00:42<00:00,  9.80it/s]


TP: 179	FN: 0	FP: 958
Recall: 1.00
Precision: 0.16
F1-score: 0.27


100%|██████████| 420/420 [00:43<00:00,  9.70it/s]


...

TP: 179	FN: 0	FP: 316
Recall: 1.00
Precision: 0.36
F1-score: 0.53


100%|██████████| 420/420 [00:43<00:00,  9.59it/s]


TP: 179	FN: 0	FP: 167
Recall: 1.00
Precision: 0.52
F1-score: 0.68


100%|██████████| 420/420 [00:43<00:00,  9.60it/s]

TP: 179	FN: 0	FP: 132
Recall: 1.00
Precision: 0.58
F1-score: 0.73


In [26]:
print(recall)
print(precision)
print(f1_score)

['1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00']
['0.14', '0.16', '0.18', '0.20', '0.22', '0.25', '0.30', '0.36', '0.52', '0.58']
['0.24', '0.27', '0.30', '0.33', '0.36', '0.40', '0.46', '0.53', '0.68', '0.73']


### NO-FALL DETECTION

In [27]:
recall, precision, f1_score = [],[],[]
for thr in range(0,10,1):
    tp, fp, fn = classifier_performance(test_dataset, model_r, device, 1,float(thr/10))
    rec = tp / (tp+fn)
    prec = tp/ (tp+fp)
    f1_s = 2/((1/rec)+(1/prec))
    recall.append(f"{rec:.2f}")
    precision.append(f"{prec:.2f}")
    f1_score.append(f"{f1_s:.2f}")
    print(f"TP: {tp}\tFN: {fn}\tFP: {fp}")
    print(f"Recall: {rec:.2f}")
    print(f"Precision: {prec:.2f}")
    print(f"F1-score: {f1_s:.2f}")

100%|██████████| 420/420 [00:42<00:00,  9.92it/s]


TP: 242	FN: 0	FP: 731
Recall: 1.00
Precision: 0.25
F1-score: 0.40


100%|██████████| 420/420 [00:42<00:00,  9.94it/s]


TP: 242	FN: 0	FP: 581
Recall: 1.00
Precision: 0.29
F1-score: 0.45


100%|██████████| 420/420 [00:42<00:00,  9.79it/s]


...

TP: 242	FN: 0	FP: 21
Recall: 1.00
Precision: 0.92
F1-score: 0.96


100%|██████████| 420/420 [00:43<00:00,  9.58it/s]


TP: 242	FN: 0	FP: 19
Recall: 1.00
Precision: 0.93
F1-score: 0.96


100%|██████████| 420/420 [00:43<00:00,  9.58it/s]

TP: 219	FN: 23	FP: 9
Recall: 0.90
Precision: 0.96
F1-score: 0.93


In [28]:
print(recall)
print(precision)
print(f1_score)

['1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '0.90']
['0.25', '0.29', '0.34', '0.41', '0.51', '0.60', '0.80', '0.92', '0.93', '0.96']
['0.40', '0.45', '0.51', '0.58', '0.68', '0.75', '0.89', '0.96', '0.96', '0.93']


## ONLY VIRTUAL

In [44]:
evaluate(model_v, test_data_loader, device=device)

creating index...
index created!
Test:  [  0/105]  eta: 0:00:40  model_time: 0.3267 (0.3267)  evaluator_time: 0.0072 (0.0072)  time: 0.3893  data: 0.0401  max mem: 2030
Test:  [100/105]  eta: 0:00:01  model_time: 0.3184 (0.3174)  evaluator_time: 0.0049 (0.0055)  time: 0.3718  data: 0.0455  max mem: 2030
Test:  [104/105]  eta: 0:00:00  model_time: 0.3167 (0.3173)  evaluator_time: 0.0047 (0.0055)  time: 0.3703  data: 0.0451  max mem: 2030
Test: Total time: 0:00:39 (0.3737 s / it)
Averaged stats: model_time: 0.3167 (0.3173)  evaluator_time: 0.0047 (0.0055)
Accumulating evaluation results...
DONE (t=0.11s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.472
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.808
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.477
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=me

### FALL DETECTION

In [45]:
recall, precision, f1_score = [],[],[]
for thr in range(0,10,1):
    tp, fp, fn = classifier_performance(test_dataset, model_v, device, 2,float(thr/10))
    rec = tp / (tp+fn)
    prec = tp/ (tp+fp)
    f1_s = 2/((1/rec)+(1/prec))
    recall.append(f"{rec:.2f}")
    precision.append(f"{prec:.2f}")
    f1_score.append(f"{f1_s:.2f}")
    print(f"TP: {tp}\tFN: {fn}\tFP: {fp}")
    print(f"Recall: {rec:.2f}")
    print(f"Precision: {prec:.2f}")
    print(f"F1-score: {f1_s:.2f}")

100%|██████████| 420/420 [00:41<00:00, 10.08it/s]


TP: 177	FN: 2	FP: 597
Recall: 0.99
Precision: 0.23
F1-score: 0.37


100%|██████████| 420/420 [00:42<00:00, 10.00it/s]


TP: 170	FN: 9	FP: 204
Recall: 0.95
Precision: 0.45
F1-score: 0.61


100%|██████████| 420/420 [00:42<00:00,  9.88it/s]


...

TP: 4	FN: 175	FP: 1
Recall: 0.02
Precision: 0.80
F1-score: 0.04


100%|██████████| 420/420 [00:43<00:00,  9.75it/s]


TP: 1	FN: 178	FP: 0
Recall: 0.01
Precision: 1.00
F1-score: 0.01


100%|██████████| 420/420 [00:43<00:00,  9.74it/s]


ZeroDivisionError: division by zero

In [46]:
print(recall)
print(precision)
print(f1_score)

['0.99', '0.95', '0.69', '0.35', '0.15', '0.02', '0.01']
['0.23', '0.45', '0.75', '0.84', '0.90', '0.80', '1.00']
['0.37', '0.61', '0.72', '0.50', '0.25', '0.04', '0.01']


### NO-FALL DETECTION

In [47]:
recall, precision, f1_score = [],[],[]
for thr in range(0,10,1):
    tp, fp, fn = classifier_performance(test_dataset, model_v, device, 1,float(thr/10))
    rec = tp / (tp+fn)
    prec = tp/ (tp+fp)
    f1_s = 2/((1/rec)+(1/prec))
    recall.append(f"{rec:.2f}")
    precision.append(f"{prec:.2f}")
    f1_score.append(f"{f1_s:.2f}")
    print(f"TP: {tp}\tFN: {fn}\tFP: {fp}")
    print(f"Recall: {rec:.2f}")
    print(f"Precision: {prec:.2f}")
    print(f"F1-score: {f1_s:.2f}")

100%|██████████| 420/420 [00:41<00:00, 10.07it/s]


TP: 242	FN: 0	FP: 1713
Recall: 1.00
Precision: 0.12
F1-score: 0.22


100%|██████████| 420/420 [00:43<00:00,  9.75it/s]


TP: 242	FN: 0	FP: 1052
Recall: 1.00
Precision: 0.19
F1-score: 0.32


100%|██████████| 420/420 [00:43<00:00,  9.72it/s]


...

TP: 241	FN: 1	FP: 216
Recall: 1.00
Precision: 0.53
F1-score: 0.69


100%|██████████| 420/420 [00:43<00:00,  9.70it/s]


TP: 241	FN: 1	FP: 188
Recall: 1.00
Precision: 0.56
F1-score: 0.72


100%|██████████| 420/420 [00:43<00:00,  9.71it/s]

TP: 241	FN: 1	FP: 179
Recall: 1.00
Precision: 0.57
F1-score: 0.73


In [48]:
print(recall)
print(precision)
print(f1_score)

['1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00']
['0.12', '0.19', '0.28', '0.34', '0.39', '0.44', '0.49', '0.53', '0.56', '0.57']
['0.22', '0.32', '0.44', '0.51', '0.56', '0.61', '0.65', '0.69', '0.72', '0.73']


## VIRTUAL THAN REAL

In [49]:
evaluate(model_vtr, test_data_loader, device=device)

creating index...
index created!
Test:  [  0/105]  eta: 0:00:43  model_time: 0.3332 (0.3332)  evaluator_time: 0.0070 (0.0070)  time: 0.4168  data: 0.0486  max mem: 2030
Test:  [100/105]  eta: 0:00:01  model_time: 0.3369 (0.3315)  evaluator_time: 0.0047 (0.0057)  time: 0.3889  data: 0.0435  max mem: 2030
Test:  [104/105]  eta: 0:00:00  model_time: 0.3373 (0.3317)  evaluator_time: 0.0055 (0.0057)  time: 0.3899  data: 0.0441  max mem: 2030
Test: Total time: 0:00:40 (0.3874 s / it)
Averaged stats: model_time: 0.3373 (0.3317)  evaluator_time: 0.0055 (0.0057)
Accumulating evaluation results...
DONE (t=0.09s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.654
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.988
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.821
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=me

### FALL DETECTION

In [50]:
recall, precision, f1_score = [],[],[]
for thr in range(0,10,1):
    tp, fp, fn = classifier_performance(test_dataset, model_vtr, device, 2,float(thr/10))
    rec = tp / (tp+fn)
    prec = tp/ (tp+fp)
    f1_s = 2/((1/rec)+(1/prec))
    recall.append(f"{rec:.2f}")
    precision.append(f"{prec:.2f}")
    f1_score.append(f"{f1_s:.2f}")
    print(f"TP: {tp}\tFN: {fn}\tFP: {fp}")
    print(f"Recall: {rec:.2f}")
    print(f"Precision: {prec:.2f}")
    print(f"F1-score: {f1_s:.2f}")

100%|██████████| 420/420 [00:43<00:00,  9.59it/s]


TP: 179	FN: 0	FP: 1589
Recall: 1.00
Precision: 0.10
F1-score: 0.18


100%|██████████| 420/420 [00:43<00:00,  9.56it/s]


TP: 179	FN: 0	FP: 1084
Recall: 1.00
Precision: 0.14
F1-score: 0.25


100%|██████████| 420/420 [00:43<00:00,  9.60it/s]


...

TP: 179	FN: 0	FP: 60
Recall: 1.00
Precision: 0.75
F1-score: 0.86


100%|██████████| 420/420 [00:43<00:00,  9.59it/s]


TP: 179	FN: 0	FP: 25
Recall: 1.00
Precision: 0.88
F1-score: 0.93


100%|██████████| 420/420 [00:43<00:00,  9.58it/s]

TP: 177	FN: 2	FP: 1
Recall: 0.99
Precision: 0.99
F1-score: 0.99


In [51]:
print(recall)
print(precision)
print(f1_score)

['1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '0.99']
['0.10', '0.14', '0.18', '0.22', '0.31', '0.44', '0.53', '0.75', '0.88', '0.99']
['0.18', '0.25', '0.31', '0.36', '0.47', '0.61', '0.70', '0.86', '0.93', '0.99']


### NO-FALL DETECTION

In [52]:
recall, precision, f1_score = [],[],[]
for thr in range(0,10,1):
    tp, fp, fn = classifier_performance(test_dataset, model_vtr, device, 1,float(thr/10))
    rec = tp / (tp+fn)
    prec = tp/ (tp+fp)
    f1_s = 2/((1/rec)+(1/prec))
    recall.append(f"{rec:.2f}")
    precision.append(f"{prec:.2f}")
    f1_score.append(f"{f1_s:.2f}")
    print(f"TP: {tp}\tFN: {fn}\tFP: {fp}")
    print(f"Recall: {rec:.2f}")
    print(f"Precision: {prec:.2f}")
    print(f"F1-score: {f1_s:.2f}")

100%|██████████| 420/420 [00:42<00:00,  9.86it/s]


TP: 242	FN: 0	FP: 399
Recall: 1.00
Precision: 0.38
F1-score: 0.55


100%|██████████| 420/420 [00:43<00:00,  9.62it/s]


TP: 242	FN: 0	FP: 62
Recall: 1.00
Precision: 0.80
F1-score: 0.89


100%|██████████| 420/420 [00:43<00:00,  9.56it/s]


...

TP: 238	FN: 4	FP: 7
Recall: 0.98
Precision: 0.97
F1-score: 0.98


100%|██████████| 420/420 [00:43<00:00,  9.60it/s]


TP: 236	FN: 6	FP: 0
Recall: 0.98
Precision: 1.00
F1-score: 0.99


100%|██████████| 420/420 [00:43<00:00,  9.59it/s]

TP: 241	FN: 1	FP: 2
Recall: 1.00
Precision: 0.99
F1-score: 0.99


In [53]:
print(recall)
print(precision)
print(f1_score)

['1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '0.98', '0.98', '1.00']
['0.38', '0.80', '0.90', '0.92', '0.92', '0.93', '0.93', '0.97', '1.00', '0.99']
['0.55', '0.89', '0.95', '0.96', '0.96', '0.96', '0.97', '0.98', '0.99', '0.99']


## VIRTUAL AND REAL

In [54]:
evaluate(model_var, test_data_loader, device=device)

creating index...
index created!
Test:  [  0/105]  eta: 0:00:42  model_time: 0.3272 (0.3272)  evaluator_time: 0.0066 (0.0066)  time: 0.4017  data: 0.0422  max mem: 2030
Test:  [100/105]  eta: 0:00:01  model_time: 0.3304 (0.3281)  evaluator_time: 0.0049 (0.0057)  time: 0.3862  data: 0.0459  max mem: 2030
Test:  [104/105]  eta: 0:00:00  model_time: 0.3327 (0.3284)  evaluator_time: 0.0049 (0.0057)  time: 0.3866  data: 0.0453  max mem: 2030
Test: Total time: 0:00:40 (0.3856 s / it)
Averaged stats: model_time: 0.3327 (0.3284)  evaluator_time: 0.0049 (0.0057)
Accumulating evaluation results...
DONE (t=0.13s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.625
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.976
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.690
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=me

### FALL DETECTION

In [55]:
recall, precision, f1_score = [],[],[]
for thr in range(0,10,1):
    tp, fp, fn = classifier_performance(test_dataset, model_var, device, 2,float(thr/10))
    rec = tp / (tp+fn)
    prec = tp/ (tp+fp)
    f1_s = 2/((1/rec)+(1/prec))
    recall.append(f"{rec:.2f}")
    precision.append(f"{prec:.2f}")
    f1_score.append(f"{f1_s:.2f}")
    print(f"TP: {tp}\tFN: {fn}\tFP: {fp}")
    print(f"Recall: {rec:.2f}")
    print(f"Precision: {prec:.2f}")
    print(f"F1-score: {f1_s:.2f}")

100%|██████████| 420/420 [00:43<00:00,  9.71it/s]


TP: 179	FN: 0	FP: 1183
Recall: 1.00
Precision: 0.13
F1-score: 0.23


100%|██████████| 420/420 [00:43<00:00,  9.68it/s]


TP: 179	FN: 0	FP: 735
Recall: 1.00
Precision: 0.20
F1-score: 0.33


100%|██████████| 420/420 [00:43<00:00,  9.69it/s]


...

TP: 179	FN: 0	FP: 14
Recall: 1.00
Precision: 0.93
F1-score: 0.96


100%|██████████| 420/420 [00:43<00:00,  9.66it/s]


TP: 174	FN: 5	FP: 4
Recall: 0.97
Precision: 0.98
F1-score: 0.97


100%|██████████| 420/420 [00:43<00:00,  9.63it/s]

TP: 160	FN: 19	FP: 1
Recall: 0.89
Precision: 0.99
F1-score: 0.94


In [56]:
print(recall)
print(precision)
print(f1_score)

['1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '0.97', '0.89']
['0.13', '0.20', '0.35', '0.50', '0.63', '0.75', '0.86', '0.93', '0.98', '0.99']
['0.23', '0.33', '0.52', '0.66', '0.77', '0.86', '0.93', '0.96', '0.97', '0.94']


### NO-FALL DETECTION

In [57]:
recall, precision, f1_score = [],[],[]
for thr in range(0,10,1):
    tp, fp, fn = classifier_performance(test_dataset, model_var, device, 1,float(thr/10))
    rec = tp / (tp+fn)
    prec = tp/ (tp+fp)
    f1_s = 2/((1/rec)+(1/prec))
    recall.append(f"{rec:.2f}")
    precision.append(f"{prec:.2f}")
    f1_score.append(f"{f1_s:.2f}")
    print(f"TP: {tp}\tFN: {fn}\tFP: {fp}")
    print(f"Recall: {rec:.2f}")
    print(f"Precision: {prec:.2f}")
    print(f"F1-score: {f1_s:.2f}")

100%|██████████| 420/420 [00:42<00:00,  9.86it/s]


TP: 242	FN: 0	FP: 786
Recall: 1.00
Precision: 0.24
F1-score: 0.38


100%|██████████| 420/420 [00:43<00:00,  9.67it/s]


TP: 242	FN: 0	FP: 595
Recall: 1.00
Precision: 0.29
F1-score: 0.45


100%|██████████| 420/420 [00:43<00:00,  9.67it/s]


...

TP: 242	FN: 0	FP: 41
Recall: 1.00
Precision: 0.86
F1-score: 0.92


100%|██████████| 420/420 [00:43<00:00,  9.65it/s]


TP: 242	FN: 0	FP: 19
Recall: 1.00
Precision: 0.93
F1-score: 0.96


100%|██████████| 420/420 [00:43<00:00,  9.66it/s]

TP: 242	FN: 0	FP: 33
Recall: 1.00
Precision: 0.88
F1-score: 0.94


In [58]:
print(recall)
print(precision)
print(f1_score)

['1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00']
['0.24', '0.29', '0.37', '0.46', '0.54', '0.60', '0.69', '0.86', '0.93', '0.88']
['0.38', '0.45', '0.54', '0.63', '0.70', '0.75', '0.82', '0.92', '0.96', '0.94']
